In [1]:
import pandas as pd
df = pd.read_csv("glass.csv")

In [2]:
df.shape

(214, 10)

In [3]:
df.columns

Index(['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type'], dtype='object')

In [4]:
df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [5]:
#Which column is the output we want to predict?
#The type column

#Are all columns numeric?
#yes

#Is there an ID column that should not be used?
#Yes, the Id column is only an identifier and should not be used.

In [6]:
df["y"]=(df["Type"]==1).astype(int)
df = df.drop("Type",axis=1)
df.head()

#What This Step Does
#Creates a binary output column y.

#Why It Is Needed
#Logistic regression is a binary classifier.

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,y
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [7]:
x = df.drop("y",axis=1)
y = df["y"]

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(
    x,y,test_size=0.2, random_state=42
)

#What This Step Does
#Split the data into training and testing sets.

#Why It Is Needed
#We must test on unseen data.
#Testing on training data gives false confidence..

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()   
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

#What This Step Does
#Scales the features to have mean 0 and standard deviation 1.

#Why It Is Needed
#Logistic regression can be sensitive to feature scales.
#Scaling can improve convergence and performance.

In [10]:
import numpy as np
def sigmoid(z):
    return 1/(1+np.exp(-z))

#What This Step Does
#Defines the sigmoid function.  

#Why It Is Needed
#Logistic regression uses the sigmoid function to convert linear outputs into probabilities.

In [11]:
def predict_probability(X, w, b):
    z = X @ w + b
    p = sigmoid(z)
    return p

In [12]:
def loss(y,p):
    loss = -y*np.log(p) - (1-y)*np.log(1-p)
    return np.mean(loss)

#What This Step Does
#Defines the logistic loss function.

#Why It Is Needed
#The loss function measures how well the model's predictions match the true labels.

In [13]:
def update_weights(X, y, w, b, lr):
    p = predict_probability(X, w, b)
    error = p - y
    w -= lr * (X.T @ error) / len(y)
    b -= lr * np.mean(error)
    return w, b

#What This Step Does
#Performs one step of gradient descent to update the weights and bias.

#Why It Is Needed
#Gradient descent is used to minimize the loss function and find the best weights for the model.

In [14]:
w = np.zeros(x_train_scaled.shape[1])
b = 0.0
lr = 0.1
epochs = 100

for epoch in range(epochs):
    w,b = update_weights(x_train_scaled, y_train.values, w, b, lr)

#What This Step Does
#Trains the logistic regression model using gradient descent.

#Why It Is Needed
#We need to optimize the weights to minimize the loss and improve predictions.

In [15]:
def predict_label(p, threshold=0.5):
    return (p >= threshold).astype(int)

#What This Step Does
#Converts predicted probabilities into binary class labels based on a threshold.

#Why It Is Needed
#We need to convert probabilities into class predictions to evaluate accuracy and make decisions.

In [16]:
p_test = predict_probability(x_test_scaled, w, b)

y_pred_05 = predict_label(p_test, threshold=0.5)
y_pred_07 = predict_label(p_test, threshold=0.7)

In [17]:
print("Accuracy (0.5):", np.mean(y_pred_05 == y_test))
print("Accuracy (0.7):", np.mean(y_pred_07 == y_test))

Accuracy (0.5): 0.8604651162790697
Accuracy (0.7): 0.7209302325581395


Logistic regression differs from perceptron because perceptron gives a hard YES/NO decision, while logistic regression outputs a probability representing confidence. The sigmoid function is important because it converts the linear score into a smooth value between 0 and 1, preserving uncertainty near the decision boundary. However, logistic regression still creates a linear decision boundary, so it cannot solve non-linear problems such as XOR. That limitation remains unsolved in this model.